In [ ]:
#|default_exp toolloop

# Tool loop

In [ ]:
#| export
from claudette.core import *
from fastcore.utils import *

Anthropic provides an [interesting example](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/customer_service_agent.ipynb) of using tools to mock up a hypothetical ordering system. We're going to take it a step further, and show how we can dramatically simplify the process, whilst completing more complex tasks.

We'll start by defining the same mock customer/order data as in Anthropic's example, plus create a entity relationship between customers and orders:

In [ ]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

We can now define the same functions from the original example -- but note that we don't need to manually create the large JSON schema, since Claudette handles all that for us automatically from the functions directly. We'll add some extra functionality to update order details when cancelling too.

In [ ]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

We're now ready to start our chat.

In [ ]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(models[-1], tools=tools)

We'll start with the same request as Anthropic showed:

In [ ]:
r = chat('Can you tell me the email address for customer C1?')
print(r.stop_reason)
r.content

tool_use


[ToolUseBlock(id='toolu_01GaTymQyJjjtZuH5QaTUpYV', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Claude asks us to use a tool. Claudette handles that automatically by just passing back the message:

In [ ]:
r = chat(r)
contents(r)

- Retrieving customer C1


'The email address for customer C1 is john@example.com.'

Let's consider a more complex case than in the original example -- what happens if a customer wants to cancel all of their orders?

In [ ]:
chat = Chat(models[-1], tools=tools)
r = chat('Please cancel all orders for customer C1 for me.')
print(r.stop_reason)
r.content

tool_use


[TextBlock(text="Okay, let's cancel all orders for customer C1.", type='text'),
 ToolUseBlock(id='toolu_018HSZGj5THYQB7Vzmk5XadR', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

This is the start of a multi-stage tool use process. Doing it manually step by step is inconvenient, so let's write a function to handle this for us:

In [ ]:
#| exports
@patch
def toolloop(self:Chat,
             pr, # Prompt to pass to Claude
             max_steps=10, # Maximum number of tool requests to loop through
             temp=0, # Temperature
             maxtok=4096, # Maximum tokens
             stop:Optional[list[str]]=None, # Stop sequences
             show_trace:bool=False, # Print tool use steps
             **kw):
    "Add prompt `pr` to dialog and get a response from Claude, automatically following up with `tool_use` messages"
    r = self(pr, temp=temp, maxtok=maxtok, stop=stop, **kw)
    i=0
    while r.stop_reason=='tool_use' and i<max_steps:
        if show_trace: print(r)
        r = self(r, temp=temp, maxtok=maxtok, stop=stop, **kw)
        i += 1
    if show_trace: print(r)
    return r

We'll start by re-running our previous request - we shouldn't have to manually pass back the `tool_use` message any more:

In [ ]:
chat = Chat(models[-1], tools=tools)
r = chat.toolloop('Can you tell me the email address for customer C1?')
r

- Retrieving customer C1


The email address for customer C1 is john@example.com.

<details>

- id: msg_018ZS1oLcBnqXjnCjcCWcvWX
- content: [{'text': 'The email address for customer C1 is john@example.com.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 732, 'output_tokens': 19}

</details>

Let's see if it can handle the multi-stage process now -- we'll add `show_trace` to see each stage of the process:

In [ ]:
chat = Chat(models[-1], tools=tools)
r = chat.toolloop('Please cancel all orders for customer C1 for me.', show_trace=True)
r

ToolsBetaMessage(id='msg_01DTMHbL3ieoESGypnpJXwWK', content=[TextBlock(text="Okay, let's cancel all orders for customer C1.", type='text'), ToolUseBlock(id='toolu_01Ewn88rpxxgTmeo2NdpLB2j', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=In: 537; Out: 72; Total: 609)
- Retrieving customer C1
ToolsBetaMessage(id='msg_018Bwa5Kmtr1pi72qp9cDJoP', content=[TextBlock(text="Based on the customer information, it looks like there are 2 orders for customer C1:\n- Order O1 for Widget A (status: Shipped)\n- Order O2 for Gadget B (status: Processing)\n\nLet's cancel both of these orders:", type='text'), ToolUseBlock(id='toolu_01V117om2ibdDr9EUSR4eBJq', input={'order_id': 'O1'}, name='cancel_order', type='tool_use'), ToolUseBlock(id='toolu_018XmD9KZkNSmFEQijyoLkjX', input={'order_id': 'O2'}, name='cancel_order', type='tool_use')], model='claude-3-haiku-202403

The cancellation of both orders was successful. I have now cancelled all orders for customer C1.

<details>

- id: msg_018NSi4aB2jn1cPnQrZ8T5Pk
- content: [{'text': 'The cancellation of both orders was successful. I have now cancelled all orders for customer C1.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 966, 'output_tokens': 24}

</details>

OK Claude thinks the orders were cancelled -- let's check one:

In [ ]:
chat.toolloop('What is the status of order O2?')

- Retrieving order O2


The status of order O2 is now 'Cancelled'. This matches the result we got when we successfully cancelled the order earlier.

<details>

- id: msg_01VQikjVpyUK4D4CUqVeKbzt
- content: [{'text': "The status of order O2 is now 'Cancelled'. This matches the result we got when we successfully cancelled the order earlier.", 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 1129, 'output_tokens': 32}

</details>

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()